# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [1]:
import pandas as pd
import pymongo
import json

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
client = pymongo.MongoClient('mongo')
database = client['exercices']
collection = database['youtube']

In [3]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Netoyer les données

In [4]:
df_youtube.dtypes

video_id           object
title              object
channel_title      object
category_id         int64
tags               object
views               int64
likes               int64
dislikes            int64
comment_total       int64
thumbnail_link     object
date              float64
dtype: object

In [5]:
df_youtube.dropna().drop_duplicates()

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09
2,cLdxuaxaQwc,My Response,PewDiePie,22,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09
...,...,...,...,...,...,...,...,...,...,...,...
7987,xlu6i6lT_vk,How Do MASSIVE Sinkholes Form?,Life Noggin,27,sinkhole|how do sinkholes form|sinkhole in wat...,440393,14362,390,1575,https://i.ytimg.com/vi/xlu6i6lT_vk/default.jpg,22.10
7988,qRoVlH1OcI4,Trump slams Clinton for defending NFL anthem p...,Business Insider,25,Business Insider|Donald Trump|Hillary Clinton|...,55762,1265,760,1873,https://i.ytimg.com/vi/qRoVlH1OcI4/default.jpg,22.10
7989,EoejGgUNmVU,LP - Lost On You (A Night at The McKittrick Ho...,LP,10,LP|Death Valley|Other People|Lost On You|The M...,142908,7088,68,437,https://i.ytimg.com/vi/EoejGgUNmVU/default.jpg,22.10
7990,MT1CMTI0EVw,Tré Melvin @ #YouTubeBlack FanFest Washington ...,YouTube FanFest,24,YouTube FanFest|#YTFF|Washington DC|USA|YTFF|#...,24532,2148,77,0,https://i.ytimg.com/vi/MT1CMTI0EVw/default.jpg,22.10


### Importer les données

In [6]:
collection_yt = collection

In [7]:
#vidage de la collection 
collection_yt.delete_many({})

In [8]:
#creation d'une chaine de caractere 
db_youtube = df_youtube.to_json(orient = 'records')

In [9]:
from pathlib import Path
import json
Path("./yt.json").write_text(json.dumps(json.loads(db_youtube), indent=4))

5000655

In [10]:
yt_json = json.loads(db_youtube)


In [11]:
import sys
sys.getsizeof(yt_json)

69152

In [12]:
yt_json[0]

{'video_id': 'XpVt6Z1Gjjo',
 'title': '1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED YOUTUBE FOREVER!',
 'channel_title': 'Logan Paul Vlogs',
 'category_id': 24,
 'tags': 'logan paul vlog|logan paul|logan|paul|olympics|logan paul youtube|vlog|daily|comedy|hollywood|parrot|maverick|bird|maverick clothes|diamond play button|logan paul diamond play button|10M subscribers|logan paul 1 year vlogging|1 year vlog|dwarf mamba play button|logan paul history|youtube history|10M|10M plaque|youtube button|diamond button|logang|logang 4 life',
 'views': 4394029,
 'likes': 320053,
 'dislikes': 5931,
 'comment_total': 46245,
 'thumbnail_link': 'https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg',
 'date': 13.09}

In [13]:
len(yt_json)

7992

In [14]:
#remplissage de a collection a partir du csv precedent
collection.insert_many(json.loads(db_youtube))

## Question 1  

In [15]:
#recherche des titre de video de la chaine "apple"
cur = collection_yt.find({"channel_title" :  "Apple"}, {"title":1, "video_id":1})
list(cur)

[{'_id': ObjectId('61b88eea4b2e1a014f950e5c'),
  'video_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple'},
 {'_id': ObjectId('61b88eea4b2e1a014f950f26'),
  'video_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple'},
 {'_id': ObjectId('61b88eea4b2e1a014f950ffe'),
  'video_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple'},
 {'_id': ObjectId('61b88eea4b2e1a014f9510fd'),
  'video_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple'},
 {'_id': ObjectId('61b88eea4b2e1a014f9511e4'),
  'video_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple'},
 {'_id': ObjectId('61b88eea4b2e1a014f951280'),
  'video_id': 'A9k88sMyiJM',
  'title': 'Apple Watch Series 3 + Apple Music — Roll — Apple'},
 {'_id': ObjectId('61b88eea4b2e1a014f95136d'),
  'video_id': 'A9k88sMyiJM',
  'title': 'Apple Watch Series 3 + Apple Music — Roll — Apple'},
 {'_id': ObjectId('61b88eea4b2e1a014f95145d'),
  'video_id': 'A

## Question 2

In [20]:
#compter le nombre de categori différentes
cur = collection_yt.aggregate([{"$group" : {"_id" : "$category_id"}}])
len(list(cur))

16

## Question 3

## Question 4

In [25]:
#recuperer les 10 videos les plus vues
cur = collection_yt.find({}, {"title":1, "channel_title":1, "views":1}).sort([("views",-1)])[:10]
list(cur)

[{'_id': ObjectId('61b88eea4b2e1a014f9515ce'),
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'views': 41500672},
 {'_id': ObjectId('61b88eea4b2e1a014f9514f9'),
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'views': 38013692},
 {'_id': ObjectId('61b88eea4b2e1a014f950f0c'),
  'title': 'ZAYN - Dusk Till Dawn ft. Sia',
  'channel_title': 'ZaynVEVO',
  'views': 36323498},
 {'_id': ObjectId('61b88eea4b2e1a014f952a3a'),
  'title': 'Eminem Rips Donald Trump In BET Hip Hop Awards Freestyle Cypher',
  'channel_title': 'BETNetworks',
  'views': 33191594},
 {'_id': ObjectId('61b88eea4b2e1a014f952960'),
  'title': 'Eminem Rips Donald Trump In BET Hip Hop Awards Freestyle Cypher',
  'channel_title': 'BETNetworks',
  'views': 32512343},
 {'_id': ObjectId('61b88eea4b2e1a014f951626'),
  'title': 'Shakira - Perro Fiel (Official Video) ft. Nicky Jam',
  'channel_title': 'shakiraVEVO',
  'views': 32136948},
 {'_id': ObjectId('61b88eea4b2e1a014f95

## Question 5

In [26]:
#calcule de la moyenne de vue par categorie
cur = collection_yt.aggregate([{"$group" : {"_id" : "$category_id", "averageViewCatergorie" : {"$avg" : "$views"}}}])
list(cur)

[{'_id': 20, 'averageViewCatergorie': 681081.8170731707},
 {'_id': 1, 'averageViewCatergorie': 1039472.6560846561},
 {'_id': 19, 'averageViewCatergorie': 464041.0833333333},
 {'_id': 27, 'averageViewCatergorie': 547582.4700598803},
 {'_id': 25, 'averageViewCatergorie': 540955.5569823436},
 {'_id': 28, 'averageViewCatergorie': 924730.548828125},
 {'_id': 22, 'averageViewCatergorie': 971532.7573696146},
 {'_id': 2, 'averageViewCatergorie': 607693.8706896552},
 {'_id': 17, 'averageViewCatergorie': 728434.3243902439},
 {'_id': 10, 'averageViewCatergorie': 1176553.612},
 {'_id': 43, 'averageViewCatergorie': 8492.5},
 {'_id': 26, 'averageViewCatergorie': 537665.1806674339},
 {'_id': 23, 'averageViewCatergorie': 1240073.294039735},
 {'_id': 24, 'averageViewCatergorie': 1154868.4397251718},
 {'_id': 15, 'averageViewCatergorie': 651404.3879310344},
 {'_id': 29, 'averageViewCatergorie': 1110334.2142857143}]

## Question 6 

In [63]:
#chaine youtube avec la plus grande moyenne de like
cur = collection_yt.aggregate(
    [
        
        {
            "$group" : 
            {
                "_id" : "$channel_title", 
                "averageLikePerVideo" :
                    {
                        "$avg": "$likes"
                    }
            }
        },
        {
            "$sort": 
            {
                "averageLikePerVideo": -1
            }
        },
        {
            "$limit": 5
        },
    ]
)
list(cur)

[{'_id': 'ZaynVEVO', 'averageLikePerVideo': 1431683.0},
 {'_id': 'ibighit', 'averageLikePerVideo': 1371766.25},
 {'_id': 'melanie martinez', 'averageLikePerVideo': 911871.5},
 {'_id': 'BETNetworks', 'averageLikePerVideo': 769687.625},
 {'_id': 'jypentertainment', 'averageLikePerVideo': 758826.6666666666}]

In [48]:
max(list(cur))

TypeError: '>' not supported between instances of 'dict' and 'dict'